In [1]:
import datetime as dt 
import numpy as np 
import yfinance as yf
import pandas as pd 
import scipy.optimize as sc
import plotly.graph_objects as go

In [2]:
#? this gets data from yfinance and places it in a DF
def get_data(stocks, start, end ):
    stockData = yf.download(stocks, start= startDate, end= endDate)
    stockData = stockData["Close"]
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return meanReturns, covMatrix

#? this calculates the portfolio returns over time time period.
def portfolioPerformance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix,weights))) * np.sqrt(252)
    return returns, std 

#? calculates the negative sharpe ratio
def negativeSR(weights, meanReturns, covMatrix, riskFreeRate = 0):
    pReturns, pStd = portfolioPerformance(weights,meanReturns,covMatrix)
    return - (pReturns - riskFreeRate)/pStd

#? Calculates the max sharpe ratio.
def maxSR(meanReturns, covMatrix, riskFreeRate = 0, constraintSet = (0,1)):
    "Minimize the negative Sharpe Ratio by altering the weights of the portfolio" 
    # finds the amount of assets from the length of mean returns 
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    # all the sumations of the portfolio have to add up to one. 
    constraints  = ({"type": "eq", "fun": lambda x: np.sum(x) - 1})
    bound = constraintSet
    #! Can mess with this number for sector allocation 
    bounds = tuple(bound for asset in range(numAssets))
    # equal allocation for the for the assets
    result = sc.minimize(negativeSR,numAssets *[1./numAssets], args = args,
                        method = "SlSQP", bounds = bounds, constraints = constraints)
    
    return result

def portfolioVariance(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]


def minimizeVariance(meanReturns, covMatrix, constraintSet = (0,1)):
    "minimize the portfolio variance by altering the weights/allocation of assets in the portfolio"
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({"type": "eq", "fun": lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound  for asset in range(numAssets))
    result = sc.minimize(portfolioVariance, numAssets * [1./numAssets], args = args,
                        method = "SLSQP", bounds = bounds, constraints = constraints)
    return result

# this is the  stocks that are searched through Y Finance stock data 
stocks = ["AAPL","KO","MCO","OXY","XOM"]
# endDate is the current data as is
endDate = dt.datetime.now()
#startDate = Current Date - dt Delta Time 365 days #! days is good number to mess with 
startDate = endDate - dt.timedelta(days= 365)
# this takes the meanReturns and takes the covariance Matrix which stores data
meanReturns, covMatrix = get_data(stocks, startDate, endDate)

    
def portfolioReturn(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[0]

def efficientOpt(meanReturns, covMatrix, returnTarget, constraintSet = (0,1)):
    """ For each return return target we want to optimize the portfolio for min variance """
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
        
    constraints = ({"type": "eq","fun": lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget },
                    {"type": "eq","fun": lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = (bound for asset in range(numAssets))
    effOpt = sc.minimize(portfolioVariance,numAssets*[1./numAssets], args = args,
                            method = "SlSQP", bounds = bounds, constraints = constraints )
    return effOpt



def calculatedResults(meanReturns,covMatrix,riskFreeRate = 0 , constraintSet = (0,1)):
    " read in mean, cov matrix and other financial information"
    " Output, Max SR, Min Volatitlity, efficient frontier "
    # this is for the max Sharpe Ratio portfolio
    maxSR_Portfolio = maxSR(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio["x"], meanReturns, covMatrix)
    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index = meanReturns.index, columns = ["allocation"])
    maxSR_allocation.allocation = [round(i*100,0) for i in maxSR_allocation.allocation]
    
    
    # Min Volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio["x"], meanReturns, covMatrix)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index = meanReturns.index, columns = ["allocation"])
    minVol_allocation.allocation = [round(i*100,0) for i in minVol_allocation.allocation]
    
    # Efficent Frontier 
    efficientList = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 20)
    for target in targetReturns:
        efficientList.append(efficientOpt(meanReturns, covMatrix,target)['fun'])
    
    maxSR_returns, maxSR_std = round(maxSR_returns*100,2), round(maxSR_std*100,2)
    minVol_returns, minVol_std = round(minVol_returns*100,2), round(minVol_std*100,2)
        
    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns

print(calculatedResults(meanReturns,covMatrix))

[*********************100%***********************]  5 of 5 completed
(29.66, 33.56,       allocation
AAPL         0.0
KO           0.0
MCO          0.0
OXY          0.0
XOM        100.0, 6.24, 18.06,       allocation
AAPL         0.0
KO          84.0
MCO          0.0
OXY          0.0
XOM         16.0, [0.18058171186453056, 0.18119453330400856, 0.18302068755439513, 0.1860244451919594, 0.1901500073269432, 0.19532630461248002, 0.2014723658980405, 0.20850245107299098, 0.2163303929938677, 0.22487288564504843, 0.2340517024266965, 0.24379498159415194, 0.25403778397500765, 0.2647221320708412, 0.2757967144555128, 0.2872163942164673, 0.2989416253454328, 0.31093784385423184, 0.32317487382965066, 0.3356263758884222], array([0.06243976, 0.07476484, 0.08708991, 0.09941499, 0.11174007,
       0.12406515, 0.13639023, 0.1487153 , 0.16104038, 0.17336546,
       0.18569054, 0.19801561, 0.21034069, 0.22266577, 0.23499085,
       0.24731592, 0.259641  , 0.27196608, 0.28429116, 0.29661624]))


<H2>Graphing<H2>

In [4]:
def EF_graph(meanReturns,covMatrix,riskFreeRate = 0, constraintSet = (0,1)):
    """Return a graph plotting the min vol, max sr and efficent frontier"""
    maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList , targetReturns = calculatedResults(meanReturns,covMatrix,riskFreeRate,constraintSet) 
    # plot 3 traces 
   
    #!Max SR
    MaxSharpeRatio = go.Scatter(
        name = "Maximum Sharpe Ratio",
        mode = "markers",
        x =[maxSR_std],
        y = [maxSR_returns],
        marker = dict(color ="red" , size =14 ,line = dict(width =3, color = "black"))
    )
    
    #!Min Vol
    MinVol = go.Scatter(
        name = "Minimum Volatility",
        mode = "markers",
        x =[minVol_std],
        y = [minVol_returns],
        marker = dict(color ="green" , size =14 ,line = dict(width =3, color = "black"))
    )
        
    
    #!Efficent Frontier
    EF_curve = go.Scatter(
        name = "Efficent Frontier",
        mode = "lines",
        x =[round(ef_std *100, 2) for ef_std in efficientList],
        y = [round(target * 100, 2) for target in targetReturns],
        line = dict(color ="black" , width = 4, dash = "dashdot")
    )
    
    data = [MaxSharpeRatio, MinVol, EF_curve]
    
    layout = go.Layout(
        title = "Portfolio Optimization with the Efficent Frontier",
        yaxis = dict(title = "Annualized Return (%)"),
        xaxis = dict(title = "Annualized Volatility (%)"),
        showlegend = True,
        legend = dict(
            x = 0.75, y = 0, traceorder = "normal",
            bgcolor = "#E2E2E2",
            bordercolor = "black",
            borderwidth = 2),
        width = 800,
        height = 600)
    
    fig =  go.Figure(data = data, layout = layout)
    return fig.show()

EF_graph(meanReturns,covMatrix)